In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:

class Neuron(torch.nn.Module):
    def __init__(self, nin):
        super(Neuron, self).__init__()
        self.w = torch.nn.Parameter(torch.randn(nin, dtype=torch.double))
        self.b = torch.nn.Parameter(torch.randn(1, dtype=torch.double))

    def forward(self, x):
        if isinstance(x, list):
            x = torch.tensor(x, dtype=torch.double)
        act = torch.dot(self.w, x) + self.b
        return torch.tanh(act)



In [ ]:
class Layer(torch.nn.Module):
    def __init__(self, nin, nout):
        super(Layer, self).__init__()
        self.neurons = torch.nn.ModuleList([Neuron(nin) for _ in range(nout)])

    def forward(self, x):
        outs = [neuron(x) for neuron in self.neurons]
        return outs[0] if len(outs) == 1 else torch.cat(outs, dim=0)



In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, nin, nouts):
        super(MLP, self).__init__()
        sz = [nin] + nouts
        self.layers = torch.nn.ModuleList([Layer(sz[i], sz[i+1]) for i in range(len(nouts))])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


In [ ]:
# Sample data
xs = [[0.5, -1.5], [1.0, 1.0], [-0.5, -0.5]]
ys = [1.0, -1.0, 0.5]

# Create MLP
nin = len(xs[0])
nouts = [4, 4, 1]
#n = MLP(nin, nouts)

n = MLP(nin, nouts).to(device)  # Move model to device
# Convert input data to tensors and move to the device
xs = [torch.tensor(x, dtype=torch.double, device=device) for x in xs]
ys = torch.tensor(ys, dtype=torch.double, device=device)

# Now proceed with training using the adjusted model and tensors


In [ ]:
# Training loop
for k in range(1):
    # Forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    # Backward pass
    for p in n.parameters():
        if p.grad is not None:
            p.grad.zero_()
    loss.backward()

    # Update
    with torch.no_grad():
        for p in n.parameters():
            if p.grad is not None:
                p -= 0.01 * p.grad

    print(k, loss.item())



0 6.217712379724729


In [ ]:
# Training loop
while loss.item() > 10e-10:
    # Forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    # Backward pass
    for p in n.parameters():
        if p.grad is not None:
            p.grad.zero_()
    loss.backward()

    # Update
    with torch.no_grad():
        for p in n.parameters():
            if p.grad is not None:
                p -= 0.05 * p.grad

    print(loss.item())

Streaming output truncated to the last 5000 lines.
2.0928472414786787e-05
2.092719537198108e-05
2.0925918485772614e-05
2.0924641756132158e-05
2.092336518303045e-05
2.0922088766438756e-05
2.092081250632852e-05
2.0919536402671037e-05
2.0918260455436083e-05
2.0916984664595646e-05
2.0915709030121723e-05
2.0914433551984115e-05
2.0913158230154535e-05
2.0911883064604306e-05
2.0910608055304185e-05
2.090933320222522e-05
2.090805850533915e-05
2.0906783964616328e-05
2.090550958002922e-05
2.0904235351547877e-05
2.0902961279143363e-05
2.0901687362787743e-05
2.0900413602451807e-05
2.0899139998106845e-05
2.0897866549724426e-05
2.089659325727556e-05
2.0895320120731354e-05
2.089404714006264e-05
2.0892774315241695e-05
2.089150164623841e-05
2.0890229133025523e-05
2.0888956775573442e-05
2.0887684573853654e-05
2.0886412527836648e-05
2.0885140637496075e-05
2.088386890280016e-05
2.088259732372293e-05
2.0881325900234007e-05
2.088005463230502e-05
2.0878783519908516e-05
2.0877512563013656e-05
2.087624176159347e

KeyboardInterrupt: 

In [ ]:

#  Testing the Trained Network
"""
xs = [[0.5, -1.5], [1.0, 1.0], [-0.5, -0.5]]
ys = [1.0, -1.0, 0.5]
"""


n(xs[1])

tensor([-0.9989], device='cuda:0', dtype=torch.float64,
       grad_fn=<TanhBackward0>)

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4
